# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
df = pd.read_csv('homepage_actions.csv')
df.head(10)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [3]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [4]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
df.id.nunique()

6328

In [6]:
df.id.duplicated().sum()

1860

In [7]:
df[df.id.duplicated()]

,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8162,2017-01-17 23:20:35.483601,451198,control,click
8164,2017-01-17 23:47:58.209653,252195,control,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8180,2017-01-18 08:54:56.879682,615849,experiment,click


In [8]:
df[df.id == 182994]

,timestamp,id,group,action
5639,2016-12-14 13:19:00.940677,182994,control,view
5640,2016-12-14 13:20:29.564080,182994,control,click


In [9]:
df[df.id == 795585]

,timestamp,id,group,action
8181,2017-01-18 09:07:37.661143,795585,control,view
8182,2017-01-18 09:09:17.363917,795585,control,click


In [10]:
# Making sure each participant participated only once
df.id.value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [12]:
# Are there dublicated values because of those who click and then view?
# The rest are possibly just clicking. 

In [13]:
clicknum = (df[df.action == 'click']['id'].nunique())
viewnum =  (df[df.action == 'view']['id'].nunique())
clickviewnum =  len(df[df.id.duplicated()])
print("Number of clickers: {}".format(clicknum))
print("Number of viewers: {}".format(viewnum))
print("Number of viewers who also clicked: {}".format(clickviewnum))
print("Number of viewers who did NOT click: {}".format(viewnum-clicknum))

Number of clickers: 1860
Number of viewers: 6328
Number of viewers who also clicked: 1860
Number of viewers who did NOT click: 4468


In [14]:
# Making sure no duplicate IDs in the experimental and control groups

In [15]:
df[df.group=='experiment']['id'].unique()

array([804196, 434745, 507599, ..., 192060, 755912, 458115])

In [16]:
exp = df[df.group=='experiment']['id'].unique() # gives the unique ID numbers
cont = df[df.group=='control']['id'].unique()
# We want to make sure same ID does not appear in other group.
overlap = pd.Series(exp).isin(cont).any()
# OR use this:
overlapIDs= set(exp) & set(cont)
print('Any overlap between the two groups?: {}'.format(overlap))
print('Overlapped IDs the two groups?: {}'.format(overlapIDs))

Any overlap between the two groups?: False
Overlapped IDs the two groups?: set()


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [17]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [18]:
control = df[df.group =='control']
controlpivot = control.pivot(index='id', columns='action', values='count').fillna(value=0)
controlpivot.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [19]:
exp = df[df.group =='experiment']
exppivot = exp.pivot(index='id', columns='action', values='count').fillna(value=0)
exppivot.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [20]:
print("Total Clicks:\tControl: {}\tExperiment: {}".format(controlpivot.click.sum(), exppivot.click.sum()))
print("Total Views:\tControl: {}\tExperiment: {}".format(controlpivot.view.sum(), exppivot.view.sum()))


Total Clicks:	Control: 932.0	Experiment: 928.0
Total Views:	Control: 3332.0	Experiment: 2996.0


In [21]:
# WHAT IS THE AVERAGE CLICK RATE PER CONDITION?
print("Click Rate:\tControl: {}\tExperiment: {}".format(controlpivot.click.mean(), exppivot.click.mean()))
# It seems like EPX group has a higher rate of clicking. Is this significant enough?

Click Rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


In [22]:
import scipy.stats as stats

In [23]:
stats.ttest_ind(controlpivot.click, exppivot.click, equal_var = False)

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.008932805628674201)

In [ ]:
stats.ttest_ind(df_control.rate, df_experiment.rate, equal_var = False)

In [ ]:
# COULD WE RUN A CHI_SQUARE TEST INSTEAD?
# Total Clicks:	Control: 932.0	Experiment: 928.0
# Total Views:	Control: 3332.0	Experiment: 2996.0

In [30]:
# We'll just sum all 1s together to see an overall change
control_views = controlpivot.view.sum()
control_clicks = controlpivot.click.sum()

experiment_views = exppivot.view.sum()
experiment_click = exppivot.click.sum()

In [45]:
# Those who clicked also viewed first.
# experiment_views-experiment_click gives you those who only viewed without clicking

observations = np.array([experiment_click, experiment_views-experiment_click])
expectations = np.array([control_clicks, control_views-control_clicks])

# clicked versus just viewed for the experiment and control:
print('OBSERVED (experiment):', observations)
print('EXPECTED (control):', expectations)

OBSERVED (experiment): [ 928. 2068.]
EXPECTED (control): [ 932. 2400.]


In [44]:
stats.chisquare(f_obs=observations, f_exp=expectations)

Power_divergenceResult(statistic=45.943834048640916, pvalue=1.2169234221091101e-11)

In [40]:
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [ ]:
# WHY BELOW TWO ARE NOT THE SAME?

In [39]:
contingency_table = pd.crosstab(index=df["group"], columns=df["action"])
print(contingency_table)
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
chi2, p, dof, expected

action      click  view
group                  
control       932  3332
experiment    928  2996


(3.636160051233291,
 0.056537191086915774,
 1,
 array([[ 968.61748901, 3295.38251099],
        [ 891.38251099, 3032.61748901]]))

In [46]:
observations = np.array([experiment_click, experiment_views])
expectations = np.array([control_clicks, control_views])

# clicked versus just viewed for the experiment and control:
print('OBSERVED (experiment):', observations)
print('EXPECTED (control):', expectations)
stats.chisquare(f_obs=observations, f_exp=expectations)

OBSERVED (experiment): [ 928. 2996.]
EXPECTED (control): [ 932. 3332.]


Power_divergenceResult(statistic=33.89952032315072, pvalue=5.8033006040559e-09)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [24]:
control_rate = controlpivot.click.mean()
print("Observed click numbers in the Exp group: {}".format(exppivot.click.sum())) 

expected_experiment_clicks_under_null = control_rate * len(exppivot)
print("Expected click numbers in the Exp group: {}".format(expected_experiment_clicks_under_null)) 

Observed click numbers in the Exp group: 928.0
Expected click numbers in the Exp group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [25]:
# 𝑛∙𝑝(1−𝑝)
n = len(exppivot)
p = control_rate
stdev = np.sqrt(n * p * (1-p))
print(stdev)

24.568547907005815


In [27]:
actual_experiment_clicks = exppivot.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/stdev
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [28]:
import scipy.stats as stats
p_val = 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Both types of analysis lead to rejection of null, although the p values are not exactly the same. The conclusion is that the experimental page is more effective than the control**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.